In [23]:
import wfdb
import numpy as np
import random
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with annotations
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Function to balance the dataset
def balance_dataset(ecg_segments, labels, label_mapping, samples_per_class=50):
    balanced_segments = []
    balanced_labels = []
    
    for label, mapped_label in label_mapping.items():
        indices = [i for i, l in enumerate(labels) if l == label]
        
        if len(indices) >= samples_per_class:
            selected_indices = random.sample(indices, samples_per_class)
        else:
            selected_indices = indices
            
        for i in selected_indices:
            balanced_segments.append(ecg_segments[i])
            balanced_labels.append(mapped_label)
    
    return np.array(balanced_segments), np.array(balanced_labels)

# Load ECG data
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]  # Include more records if needed
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)

# Define the label mapping
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}

# Balance the dataset
balanced_ecg_segments, balanced_labels = balance_dataset(ecg_segments, labels, label_mapping, samples_per_class=50)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(balanced_labels), y=balanced_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Load noise data
em_noise, bw_noise, ma_noise = load_noise_data()

# Noise types
noises = {'EM': em_noise, 'BW': bw_noise, 'MA': ma_noise}
snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(balanced_ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    model_noisy = classify_heartbeats(features_noisy, balanced_labels)
    model_denoised = classify_heartbeats(features_denoised, balanced_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in balanced_labels
    class_names = ['N', 'V', 'A', 'L']
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(balanced_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(balanced_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy for each class in percentage
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] * 100 for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] * 100 for class_name in class_names}
    }

# Output the results for each class and noise condition in percentage
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies (in %):")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.2f}%")
    print("Denoised data accuracies (in %):")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.2f}%")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step
Noise type: EM
Noisy data accuracies (in %):
  N: 83.93%
  V: 100.00%
  A: 93.18%
  L: 100.00%
Denoised data accuracies (in %):
  N: 28.57%
  V: 80.00%
  A: 38.27%
  L: 49.00%


Noise type: BW
Noisy data accuracies (in %):
  N: 83.64%
  V: 97.96%
  A: 91.30%
  L: 100.00%
Denoised data accuracies (in %):
  N: 29.63%
  V: 58.33%
  A: 40.24%
  L: 58.23%


Noise type: MA
Noisy data accuracies (in %):
  N: 82.14%
  V: 97.96%
  A: 91.11%
  L: 100.00%
Denoised data accuracies (in %):
  N: 57.14%
  V: 75.00%
  A: 43.06%
  L: 45.28%




In [26]:
import wfdb
import numpy as np
import random
import pywt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Model

# Load ECG data with annotations
def load_ecg_data_with_labels(record_numbers, segment_length=512):
    ecg_segments = []
    labels = []
    for rec_num in record_numbers:
        record = wfdb.rdrecord(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}')
        annotation = wfdb.rdann(f'M:\\Dissertation\\New folder\\mit-bih-arrhythmia-database-1.0.0/{rec_num}', 'atr')
        
        for i in range(len(annotation.sample)):
            start = max(0, annotation.sample[i] - segment_length // 2)
            end = min(len(record.p_signal), start + segment_length)
            if end - start == segment_length:
                ecg_segments.append(record.p_signal[start:end, 0])  # Assuming MLII lead
                labels.append(annotation.symbol[i])
    
    return np.array(ecg_segments), np.array(labels)

# Function to balance the dataset
def balance_dataset(ecg_segments, labels, label_mapping, samples_per_class=50):
    balanced_segments = []
    balanced_labels = []
    
    for label, mapped_label in label_mapping.items():
        indices = [i for i, l in enumerate(labels) if l == label]
        
        if len(indices) >= samples_per_class:
            selected_indices = random.sample(indices, samples_per_class)
        else:
            selected_indices = indices
            
        for i in selected_indices:
            balanced_segments.append(ecg_segments[i])
            balanced_labels.append(mapped_label)
    
    return np.array(balanced_segments), np.array(balanced_labels)

# Load ECG data
# ecg_records = [205]  # Include more records if needed
ecg_records = [103, 105, 111, 116, 122, 205, 213, 219, 223, 230]
ecg_segments, labels = load_ecg_data_with_labels(ecg_records)

# Define the label mapping
label_mapping = {'N': 0, 'V': 1, 'A': 2, 'L': 3}

# Balance the dataset
balanced_ecg_segments, balanced_labels = balance_dataset(ecg_segments, labels, label_mapping, samples_per_class=50)

# Load noise data from the correct directory and filenames
def load_noise_data():
    em = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\em', sampfrom=0).p_signal[:, 0]
    bw = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\bw', sampfrom=0).p_signal[:, 0]
    ma = wfdb.rdrecord(r'M:\\Dissertation\\New folder\\mit-bih-noise-stress-test-database-1.0.0\\ma', sampfrom=0).p_signal[:, 0]
    return em, bw, ma

# Instantiate the generator model
def build_generator(input_shape=(512, 1)):
    inp = Input(shape=input_shape)

    x = Conv1D(64, 15, padding='same')(inp)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1D(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv1D(512, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(256, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(128, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv1DTranspose(64, 15, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    out = Conv1DTranspose(1, 15, padding='same', activation='tanh')(x)
    
    return Model(inp, out)

generator = build_generator()

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(balanced_labels), y=balanced_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Extend and add noise to ECG segments
def extend_noise_signal(noise_signal, target_length):
    repeated_noise = np.tile(noise_signal, int(np.ceil(target_length / len(noise_signal))))
    return repeated_noise[:target_length]

def calculate_snr(signal, noise):
    signal_power = np.sum(np.square(signal))
    noise_power = np.sum(np.square(noise))
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Extend and add noise to ECG segments using the SNR formula
def add_noise_to_segments(ecg_segments, noise_signal, target_snr_db):
    noisy_segments = []
    noise_signal = extend_noise_signal(noise_signal, ecg_segments.shape[1])
    
    for ecg_segment in ecg_segments:
        current_snr = calculate_snr(ecg_segment, noise_signal[:len(ecg_segment)])
        scaling_factor = np.sqrt(np.sum(np.square(ecg_segment)) / (np.sum(np.square(noise_signal)) * 10**(target_snr_db / 10)))
        scaled_noise = noise_signal[:len(ecg_segment)] * scaling_factor
        noisy_segment = ecg_segment + scaled_noise
        noisy_segments.append(noisy_segment)
    return np.array(noisy_segments)

# Denoise function
def denoise_signal(generator, noisy_signal):
    noisy_signal = np.expand_dims(noisy_signal, axis=-1)
    denoised_signal = generator.predict(noisy_signal)
    return denoised_signal.squeeze()

# Function to extract wavelet features
def extract_wavelet_features(ecg_slice):
    coeffs = pywt.wavedec(ecg_slice, 'db6', level=5)
    return coeffs[0]  # You may want to use more features from different levels

# Function to classify heartbeats using SVM
def classify_heartbeats(features, labels):
    clf = SVC(kernel='linear', class_weight=class_weight_dict)
    clf.fit(features, labels)
    return clf

# Load noise data
em_noise, bw_noise, ma_noise = load_noise_data()

# Create combined noise types
combined_noises = {
    'EM+MA': em_noise + ma_noise,
    'EM+BW': em_noise + bw_noise,
    'MA+BW': ma_noise + bw_noise,
    'EM+BW+MA': em_noise + bw_noise + ma_noise
}

# Add single noises to combined noises
noises = {
    'EM': em_noise,
    'BW': bw_noise,
    'MA': ma_noise
}
noises.update(combined_noises)

snr_db = 0  # Example SNR value

results = {}

for noise_name, noise_data in noises.items():
    noisy_ecg_slices = add_noise_to_segments(balanced_ecg_segments, noise_data, snr_db)
    denoised_ecg_slices = denoise_signal(generator, noisy_ecg_slices)
    
    features_noisy = np.array([extract_wavelet_features(slice) for slice in noisy_ecg_slices])
    features_denoised = np.array([extract_wavelet_features(slice) for slice in denoised_ecg_slices])
    
    model_noisy = classify_heartbeats(features_noisy, balanced_labels)
    model_denoised = classify_heartbeats(features_denoised, balanced_labels)
    
    predictions_noisy = model_noisy.predict(features_noisy)
    predictions_denoised = model_denoised.predict(features_denoised)
    
    # Get the unique classes present in balanced_labels
    class_names = ['N', 'V', 'A', 'L']
    
    # Evaluate accuracy for each class
    report_noisy = classification_report(balanced_labels, predictions_noisy, target_names=class_names, output_dict=True, zero_division=0)
    report_denoised = classification_report(balanced_labels, predictions_denoised, target_names=class_names, output_dict=True, zero_division=0)
    
    # Store accuracy for each class in percentage
    results[noise_name] = {
        'noisy': {class_name: report_noisy[class_name]['precision'] * 100 for class_name in class_names},
        'denoised': {class_name: report_denoised[class_name]['precision'] * 100 for class_name in class_names}
    }

# Output the results for each class and noise condition in percentage
for noise_name, metrics in results.items():
    print(f"Noise type: {noise_name}")
    print("Noisy data accuracies (in %):")
    for class_label, accuracy in metrics['noisy'].items():
        print(f"  {class_label}: {accuracy:.2f}%")
    print("Denoised data accuracies (in %):")
    for class_label, accuracy in metrics['denoised'].items():
        print(f"  {class_label}: {accuracy:.2f}%")
    print("\n")


C:\Python3\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 422ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 396ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 401ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 409ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 389ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 391ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step
Noise type: EM
Noisy data accuracies (in %):
  N: 90.00%
  V: 97.96%
  A: 88.24%
  L: 100.00%
Denoised data accuracies (in %):
  N: 20.59%
  V: 54.84%
  A: 50.00%
  L: 51.61%


Noise type: BW
Noisy data accuracies (in %):
  N: 88.46%
  V: 98.00%
  A: 91.67%
  L: 100.00%
Denoised data accuracies (in %):
  N: 23.17%
  V: 52.17%
  A: 50.00%
  L: 57.83%


Noise type: MA
Noisy data accuracies (in %):
  N: 93.75%
  V: 96.00%
  A: 88.46%
  L: 100.00%
Denoised data accuracies (in %):
  N: 57.14%
  V: 68.97%
  A: 56.00%
  L: 40.00%


Noise type: EM+MA
Noisy data accuracies (in %):
  N: 90.00%
  V: 97.96%
  A: 88.24%
  L: 100.00%
Denoised data accuracies (in %):
  N: 20.83%
  V: 53.57%
  A: 22.22%
  L: 52.75%


Noise type: EM+B